# 将模型转换成 ONNX

In [2]:
import os

import onnx
import onnxruntime as ort
import paddle2onnx
from onnxconverter_common import float16

In [3]:
onnx_dir = r"G:\code\github\PaddleNLP\outputs\tnews\export_model\onnx"
os.makedirs(onnx_dir, exist_ok=True)

float_onnx_file = os.path.join(onnx_dir, "model.onnx")

In [4]:
_static_model_file = r"G:\code\github\PaddleNLP\outputs\tnews\export_model\model.pdmodel"
_static_params_file = r"G:\code\github\PaddleNLP\outputs\tnews\export_model\model.pdiparams"

onnx_model = paddle2onnx.command.c_paddle_to_onnx(
    model_file=_static_model_file,
    params_file=_static_params_file,
    opset_version=13,
    enable_onnx_checker=True,
)
with open(float_onnx_file, "wb") as f:
    f.write(onnx_model)

In [ ]:
# 可能有数值溢出, fp16 精度不够
# fp16_model_file = os.path.join(onnx_dir, "fp16_model.onnx")
# onnx_model = onnx.load_model(float_onnx_file)
# trans_model = float16.convert_float_to_float16(onnx_model, keep_io_types=True)
# onnx.save_model(trans_model, fp16_model_file)

In [6]:
providers = [("CUDAExecutionProvider", {"device_id": 0})]
sess_options = ort.SessionOptions()
sess_options.intra_op_num_threads = os.cpu_count() // 2
sess_options.inter_op_num_threads = os.cpu_count() // 2
# 原来是在这里初始化 predictor 的, 而且用的是 fp16 版本的
predictor = ort.InferenceSession(float_onnx_file, sess_options=sess_options, providers=providers)

In [7]:
print(predictor.get_providers())

input_handler = [i.name for i in predictor.get_inputs()]
print(input_handler)

['CUDAExecutionProvider', 'CPUExecutionProvider']
['input_ids', 'token_type_ids', 'position_ids', 'attention_mask', 'omask_positions', 'cls_positions']


# 好了, 转换成 onnx 模型已经可以了, 现在是要准备输入数据了